## Persona

👤 Persona : Léna, écologue spécialisée en biodiversité antarctique
Profil :
Âge : 34 ans

Fonction : Chargée de recherche dans un institut de conservation de la faune sauvage.

Outils utilisés : Python, Databricks, R, QGIS

Objectif : Étudier l’évolution morphologique des manchots (penguins) en Antarctique selon les espèces, les îles d’observation et les années, dans un contexte de changement climatique.

Besoins :
Visualiser les différences morphologiques selon les espèces (Adélie, Gentoo, Chinstrap).

Étudier les relations entre les caractéristiques physiques : taille du bec, nageoires, poids.

Comparer les individus selon leur sexe et leur localisation (île).

Observer l’évolution interannuelle (par année) des mesures.

Détecter des tendances ou des anomalies morphologiques liées au réchauffement ou à la nutrition.

📊 Indicateurs pertinents pour Léna :
Poids moyen par espèce et par sexe

Longueur moyenne du bec selon l’île

Comparaison graphique des flippers par espèce

Corrélation entre bill_length_mm et body_mass_g

Évolution annuelle de la morphologie moyenne par espèce

Répartition des individus par sexe et île

## Préparation des données

In [18]:
case class Penguin(
  species: String,
  island: String,
  bill_length_mm: Option[Double],
  bill_depth_mm: Option[Double],
  flipper_length_mm: Option[Int],
  body_mass_g: Option[Int],
  sex: String,
  year: Int
)


defined class Penguin


## récupération des données

In [ ]:

val data = sc.textFile("../csv/penguins.csv")
  .mapPartitionsWithIndex { (idx, iter) =>
    val lines = if (idx == 0) iter.drop(1) else iter
    lines.map { line =>
      val cols = line.split(",", -1).map(_.trim)
      Penguin(
        species = cols(1),
        island = cols(2),
        bill_length_mm = scala.util.Try(cols(3).toDouble).toOption,
        bill_depth_mm = scala.util.Try(cols(4).toDouble).toOption,
        flipper_length_mm = scala.util.Try(cols(5).toInt).toOption,
        body_mass_g = scala.util.Try(cols(6).toInt).toOption,
        sex = cols(7),
        year = scala.util.Try(cols(8).toInt).getOrElse(0)
      )
    }
  }.toDF()

  data.show(100)



+-------+---------+--------------+-------------+-----------------+-----------+------+----+
|species|   island|bill_length_mm|bill_depth_mm|flipper_length_mm|body_mass_g|   sex|year|
+-------+---------+--------------+-------------+-----------------+-----------+------+----+
| Adelie|Torgersen|          39.1|         18.7|              181|       3750|  male|2007|
| Adelie|Torgersen|          39.5|         17.4|              186|       3800|female|2007|
| Adelie|Torgersen|          40.3|         18.0|              195|       3250|female|2007|
| Adelie|Torgersen|          NULL|         NULL|             NULL|       NULL|      |2007|
| Adelie|Torgersen|          36.7|         19.3|              193|       3450|female|2007|
| Adelie|Torgersen|          39.3|         20.6|              190|       3650|  male|2007|
| Adelie|Torgersen|          38.9|         17.8|              181|       3625|female|2007|
| Adelie|Torgersen|          39.2|         19.6|              195|       4675|  male|2007|

data = [species: string, island: string ... 6 more fields]


[species: string, island: string ... 6 more fields]